# Consignes pour le TP spaCY

## Objectifs

L'objectif du TP est d'utiliser le [matcher](https://spacy.io/api/matcher) de spaCy pour reconnaître des variantes de syntagmes nominaux dans un texte en français.
Pour ce faire, vous pouvez vous aider des notebooks utilisés lors de la séance ainsi que de la [documentation](https://spacy.io/usage/rule-based-matching) en ligne.

## Prise en main de l'outil

Commençons par importer la librairie spaCy ainsi que le module Matcher : 

In [ ]:
import spacy
from spacy.matcher import Matcher

Partons ensuite d'un texte d'exemple en français tiré de [Wikipédia](https://fr.wikipedia.org/wiki/ChatGPT) :

In [ ]:
text = """ChatGPT (Chat Generative Pre-trained Transformer) est un prototype d'agent conversationnel utilisant l'intelligence artificielle, développé par OpenAI et spécialisé dans le dialogue.
L'agent conversationnel de ChatGPT repose sur le modèle de langage GPT d'OpenAI, et est affiné en continu grâce à l'utilisation de techniques d'apprentissage supervisé et d'apprentissage par renforcement.
ChatGPT est capable de générer des réponses à des questions, de compléter des phrases, de traduire des textes, d'écrire des articles et de tenir des conversations avec des humains. Il peut également synthétiser des textes suivant un ensemble de contraintes, telles que le ton, le style et le sujet. Il a été utilisé dans diverses applications, telles que la génération de sous-titres pour des vidéos et la création de chatbots.
En raison de ses capacités multiples, le prototype suscite également des inquiétudes. Sont craints des détournements à des fins malveillantes, des risques de plagiat dans le monde académique et de possibles suppressions d'emplois dans certains secteurs. ChatGPT soulève également des préoccupations en matière de sécurité et de confidentialité, car le modèle peut être utilisé pour générer des faux textes et des informations trompeuses.
Lancé en novembre 2022 dans une version gratuite et non connectée à Internet, ChatGPT bénéficie d’une large exposition médiatique et reçoit un accueil globalement positif, bien que son exactitude factuelle soit critiquée.
En janvier 2023, ChatGPT compte plus de 100 millions de comptes enregistrés."""

Téléchargeons le modèle spaCy pour le français (fr), de petite taille (sm = small) :

In [ ]:
!python -m spacy download fr_core_news_sm

Ce modèle peut à présent être chargé en mémoire, et le texte converti en objet spaCy : 

In [ ]:
nlp = spacy.load("fr_core_news_sm")
doc = nlp(text)

Définissons un pattern qui indique que l'on recherche tous les mots dont le Part-of-Speech (POS) est de type "nom commun" (NOUN) : 

In [ ]:
# un premier pattern très simple
pattern1 = [[{"POS": "NOUN"}]]

Regardons ce que spaCy trouve comme matches (noms) dans notre texte : 

In [ ]:
def match_pattern(pattern):
    """Take a pattern as input and return matches"""
    matcher = Matcher(nlp.vocab)
    matcher.add("", pattern)
    matches = matcher(doc)
    for _, start, end in matches:
        span = doc[start:end]
        print(span.text)

match_pattern(pattern1)

spaCy trouve une soixantaine de noms, dont quelques faux positifs.

Essayons à présent un pattern légèrement plus complexe : 

In [ ]:
pattern2 = [[
            {"POS": "DET"}, 
            {"POS": "NOUN"},
            {"POS": "ADJ"}
           ]]

match_pattern(pattern2)

Il est possible de rendre certains éléments optionnels ou de les répéter avec les opérateurs ? (0 ou 1), * (0 ou plus) et + (1 ou plus) ainsi que {1} (exactement 1) etc.

In [ ]:
pattern3 = [[
            {"POS": "DET", "OP": "?"}, 
            {"POS": "NOUN", "OP": "{1}"},
            {"POS": "ADJ", "OP": "+"}
           ]]

match_pattern(pattern3)

Vous remarquerez que le SN est matché avec ET sans déterminant ce qui crée de la redondance, mais cela permet en même temps de trouver des matches supplémentaires comme "informations trompeuses".

Il est également possible de fournir une liste de patterns : 

In [ ]:
patterns = [
    [{"POS": "NOUN"}, {"POS": "ADJ"}],
    [{"POS": "ADJ"}, {"POS": "NOUN"}],
    [{"POS": "PROPN"}]
]

match_pattern(patterns)

Ceci permet d'inclure également les adjectifs placés avant le nom, ainsi que les noms propres.

## Travail à remettre

Enrichissez la liste de patterns pour couvrir un maximum de syntagmes nominaux valides, tout en excluant les autres constructions syntaxiques. Il faut donc trouver un équilibre en bruit et silence pour minimiser à la fois les faux négatifs (syntagmes valides mais non reconnus par le matcher) et les faux positifs (matches reconnus par erreur qui viennent "polluer" la liste des résultats).

Vous pouvez travailler directement dans la copie du notebook enregistrée sur votre Google Drive, ou bien dans un fichier local. N'hésitez pas à modifier le texte de départ pour varier les constructions possibles, et à expérimenter avec les modèles `fr_core_news_md` (medium) et `fr_core_news_lg` (large) pour voir si cela améliore les résultats.

Quand vous avez terminé, exportez la liste de patterns au format JSON grâce à la cellule ci-dessous. Rédigez ensuite un court rapport (1-2 pages maximum) en PDF pour expliquer le contexte du TP, votre démarche ainsi que les choix effectués.

Les deux fichiers sont à remettre sur l'UV pour le **mardi 23 mai** au plus tard. Vous serez évalués sur l'efficacité de vos patterns (rappel et précision), mais aussi sur votre réflexion.

In [ ]:
from google.colab import files
import json

json_dump = json.dumps(patterns)
with open('tp_spacy.json', 'w') as f:
  f.write(json_dump)

files.download('tp_spacy.json')